## APRS signals heard on the International Space Station

This notebook parses the log of messages sent via ISS on the ARISS website and displays the log entries on the map. The messages are parsed and displayed as well.

Let's start with the libs we need - folium for the map display and bs4 for the website parsing

In [ ]:
!python3 -m pip install folium
!python3 -m pip install requests
!python3 -m pip install beautifulsoup4

In [ ]:
import folium
import requests
import datetime
from bs4 import BeautifulSoup

Let's download the log page and get the info to display on the map

In [ ]:
html_doc = requests.get("http://www.ariss.net/index.cgi?absolute=1").text

soup = BeautifulSoup(html_doc, 'html.parser')
table = soup.findAll("table")[0]
table_rows = table.find_all('tr')
table_rows.pop(0)

The `final` dict lets us accumulate the result while we are going through the ARISS website.
<br>
<br>
The info extracted is the station's callsign, icon, geographical coordinates and date the message was heard.
Besides the info mentioned above, the list of messages is extracted from the findu.com. The data row is populated with the message sender and recipient, as well as, the text of the message

In [16]:
final = {}
for tr in table_rows:
    data = {}
    td = tr.find_all('td')
    data["call"] = td[0].text.replace(" ", "")
    data["img"] = td[0].find_all("img")[0]["src"]

    try:
        page_messages = requests.get("http://www.findu.com/cgi-bin/msg.cgi?" + data["call"]).text
        soup = BeautifulSoup(page_messages, 'html.parser')
        rows_messages = soup.findAll("table")[0].find_all('tr')
        rows_messages.pop(0)

        data["messages"] = []
        for i in rows_messages:
            t = i.find_all('td')
            data["messages"].append({"from": t[0].text.replace(" ", ""), "to": t[1].text.replace(" ", ""), "text": t[4].text})

    except Exception:
         data["messages"] = []

    try:
        data["lat"] = float(td[2].text)
        data["lon"] = float(td[3].text)
    except:
        data["lat"] = 0
        data["lon"] = 0
    
    data["UTC heard"] = datetime.datetime.strptime(td[4].text.replace(" ", ""), "%Y%m%d%H%M%S")
    final[data["call"]] = data

Now we initialize the folium.Map to display the locations of the messages were sent/received to/from

In [17]:
m = folium.Map(
    location=[0, 0],
    zoom_start=2,
)

for call, row in final.items():
    pup = '<i>{0} {1}</i>'.format(call, row["UTC heard"])
    icon = folium.features.CustomIcon(row["img"], icon_size=(20, 20))
    folium.Marker([row["lat"], row["lon"]], popup=pup, icon=icon).add_to(m)

# Let's see what we've got
m

Lovely, so many hams are chatting right now via the ISS! Let's see what the might be up to

In [18]:
final['W4AEJ']["messages"]

[{'from': 'KB5CZH', 'to': 'W4AEJ', 'text': ' QSL de KB5CZH, and 73 via ISS'},
 {'from': 'KB5CZH', 'to': 'W4AEJ', 'text': ' Hello from Texas EM11 (Live)'},
 {'from': 'W4AEJ', 'to': 'K7ZOO', 'text': ' Hello from Niceville FL'},
 {'from': 'KE5TAY-5', 'to': 'W4AEJ', 'text': ' Hello from Texas EM20 qsl?{2'},
 {'from': 'W4AEJ', 'to': 'K5DNA', 'text': ' QSL.  73 de Dale'},
 {'from': 'K5DNA', 'to': 'W4AEJ', 'text': ' 73s from EM35{48'},
 {'from': 'K4KDR-6', 'to': 'W4AEJ', 'text': ' QSL - 73!!'},
 {'from': 'K4KDR-6',
  'to': 'W4AEJ',
  'text': ' Heard you via ISS in Montpelier, VA FM17es'},
 {'from': 'N1RCN', 'to': 'W4AEJ', 'text': ' Gotcha!'},
 {'from': 'KD6RF', 'to': 'W4AEJ', 'text': ' CQ CQ CQ All - EM22lr Texas'},
 {'from': 'N1RCN', 'to': 'W4AEJ', 'text': ' Hello from EL87 (Live)'},
 {'from': 'KD6RF', 'to': 'W4AEJ', 'text': ' QSL tnx and 73'},
 {'from': 'KD6RF',
  'to': 'W4AEJ',
  'text': ' QSL from Dave in EM22lr Gilmer Texas'},
 {'from': 'N8URE-7', 'to': 'W4AEJ', 'text': ' QSL el95{13'},
